In [11]:
import requests
import os
from bs4 import BeautifulSoup
import re

import urllib.parse

In [12]:
siteUrl = "https://classiccars.com/listings/find"
siteUrl = "?".join([siteUrl, "auction=false"])
siteUrl = "&".join([siteUrl, "dealer=false"])
siteUrl = "&".join([siteUrl, "private=true"])
siteUrl = "&".join([siteUrl, "ps=60"])
print(siteUrl)

https://classiccars.com/listings/find?auction=false&dealer=false&private=true&ps=60


In [13]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=siteUrl)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

In [22]:
#<p class="search-result-info">Now showing page 1 of 5. 60 results per page.</p>
max_page_css = "#search-results-header > div.hidden-xs > p:nth-of-type(2)"
max_page_selector = tagPageSoup.select(max_page_css)
max_page_string = max_page_selector[0].get_text()
match = re.search(r'Now showing page \d+ of (\d+)\.', max_page_string)
if match:
    max_page = int(match.group(1))
else:
    max_page = 0
print(max_page)

5


In [30]:
from random import randint

i = randint(0,max_page)
tagPageUrl = siteUrl+"&p="+str(i)
print(tagPageUrl)

# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

links = tagPageSoup.find_all("a", class_="img-cell")
print(len(links))

link = links[randint(0,len(links)-1)]
url = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', link['href']))
print(url)

https://classiccars.com/listings/find?auction=false&dealer=false&private=true&ps=60&p=3
60
https://classiccars.com/listings/view/900526/1996-mercedes-benz-sl500-for-sale-in-ft-myers-florida-33908


In [31]:
page = requests.get(url=url)
soup = BeautifulSoup(page.content, 'lxml')

model = soup.find_all("h1")
if(len(model)):
    model = model[0].get_text()
else:
    model = "Unknown"
print(model)

price = soup.select('span.seller-price-label > span:nth-of-type(1)')
if(len(price)):
    price = price[0].get_text()
else:
    price = "Unknown"
print(price)

description_css = "#listing-detail-content > div.vehicle-description > p:nth-of-type(2)"
description = soup.select(description_css)
if(len(description)):
    description = description[0].get_text()
else:
    description = ""
if(len(description)):
    match = re.search(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}", description)
    if match:
        phone = match.group()
    else:
        phone = ""
print(phone)

For Sale: 1996 Mercedes-Benz SL500 in Ft Myers, Florida
$18,000 (OBO)



In [32]:
for i in range(1, max_page):
    tagPageUrl = siteUrl+"&p="+str(i)

    # Retrieve the page with tag results and set it up to be scraped
    tagPage = requests.get(url=tagPageUrl)
    tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

    links = tagPageSoup.find_all("a", class_="img-cell")
    for link in links:
        url = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', link['href']))
        try:
            page = requests.get(url=url)
            soup = BeautifulSoup(page.content, 'lxml')

            model = soup.find_all("h1")
            if(len(model)):
                model = model[0].get_text()
            else:
                model = "Unknown"
            #print(model)

            price = soup.select('span.seller-price-label > span:nth-of-type(1)')
            if(len(price)):
                price = price[0].get_text()
            else:
                price = "Unknown"
            #print(price)

            description_css = "#listing-detail-content > div.vehicle-description > p:nth-of-type(2)"
            description = soup.select(description_css)
            if(len(description)):
                description = description[0].get_text()
            else:
                description = ""
            if(len(description)):
                match = re.search(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}", description)
                if match:
                    phone = match.group()
                else:
                    phone = ""
                if len(phone):
                    bigFile = open('classiccars.txt', 'a')
                    bigFile.write(model + '\t' + price + '\t' + phone + '\n')
                    bigFile.close()
        except:
            pass